# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation():
    local_nhl = nhl_df.copy()
    local_nhl = local_nhl[(~local_nhl["team"].str.contains("Division")) & (local_nhl["year"].eq(2018))]
    local_nhl["team"] = local_nhl["team"].str.replace("*", "")
    local_nhl[["W", "L"]] = local_nhl[["W", "L"]].astype(int)
    local_nhl["win_loss"] = local_nhl["W"] / (local_nhl["W"] + local_nhl["L"])
    local_nhl = local_nhl[["team", "win_loss"]]
    #print(local_nhl.head(30))
    
    local_cities = cities.copy()
    local_cities = local_cities.replace(r"(\[.*\])", "", regex=True).replace(r"(\s*\—\s*)", "", regex=True).replace("", np.nan)
    
    def team_list(team_string):
        team_pattern = "((?:[0-9]+|[A-Z])[a-z]+(?:\s(?:[0-9]+|[A-Z])[a-z]+)*)"
        return list(re.findall(team_pattern, team_string))
    
    local_cities["NHL"] = local_cities["NHL"].apply(lambda x: team_list(x) if not pd.isna(x) else np.nan)
    local_cities = local_cities.explode("NHL").reset_index(drop=True)
    local_cities.drop(["NFL", "MLB", "NBA"], axis=1, inplace=True)
    local_cities.dropna(subset=["NHL"], how="all", inplace=True, axis=0)
    #print(local_cities)
    
    known_teams = list(local_cities[~pd.isna(local_cities["NHL"])]["NHL"])
    local_nhl["team_name"] = local_nhl["team"].apply(lambda x: [y for y in known_teams if y in x])
    local_nhl["team_name"] = local_nhl["team_name"].apply(lambda x: x[0])
    
    merged_df = local_cities.merge(local_nhl, left_on="NHL", right_on="team_name", how="left")
    merged_df["Population (2016 est.)[8]"] = merged_df["Population (2016 est.)[8]"].astype(int)
    
    result_df = merged_df.groupby("Metropolitan area").agg({"Population (2016 est.)[8]": "mean", "win_loss": "mean"})
    # print(merged_df[["Metropolitan area", "Population (2016 est.)[8]", "win_loss"]])
    # print(result_df)
    
    population_by_region = result_df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = result_df["win_loss"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    local_nba = nba_df.copy()
    local_nba = local_nba[(~local_nba["team"].str.contains("Division")) & (local_nba["year"].eq(2018))]
    local_nba["team"] = local_nba["team"].replace("\s\(.*\)", "", regex=True).str.replace("*", "")
    local_nba[["W", "L"]] = local_nba[["W", "L"]].astype(int)
    local_nba["win_loss"] = local_nba["W"] / (local_nba["W"] + local_nba["L"])
    local_nba = local_nba[["team", "win_loss"]]
    # print(local_nba.head(30))

    local_cities = cities.copy()
    local_cities = local_cities.replace(r"(\[.*\])", "", regex=True).replace(r"(\s*\—\s*)", "", regex=True).replace("", np.nan)
    
    def team_list(team_string):
        team_pattern = "((?:[0-9]+|[A-Z])[a-z]+(?:\s(?:[0-9]+|[A-Z])[a-z]+)*)"
        return list(re.findall(team_pattern, team_string))
    
    local_cities["NBA"] = local_cities["NBA"].apply(lambda x: team_list(x) if not pd.isna(x) else np.nan)
    local_cities = local_cities.explode("NBA").reset_index(drop=True)
    local_cities.drop(["NFL", "MLB", "NHL"], axis=1, inplace=True)
    local_cities.dropna(subset=["NBA"], how="all", inplace=True, axis=0)
    # print(local_cities)
    
    known_teams = list(local_cities[~pd.isna(local_cities["NBA"])]["NBA"])
    local_nba["team_name"] = local_nba["team"].apply(lambda x: [y for y in known_teams if y in x])
    local_nba["team_name"] = local_nba["team_name"].apply(lambda x: x[0])
    
    merged_df = local_cities.merge(local_nba, left_on="NBA", right_on="team_name", how="left")
    merged_df["Population (2016 est.)[8]"] = merged_df["Population (2016 est.)[8]"].astype(int)
    
    result_df = merged_df.groupby("Metropolitan area").agg({"Population (2016 est.)[8]": "mean", "win_loss": "mean"})
    # print(merged_df[["Metropolitan area", "Population (2016 est.)[8]", "win_loss", "team"]])
    # print(result_df)
        
    population_by_region = result_df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = result_df["win_loss"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    local_mlb = mlb_df.copy()
    local_mlb = local_mlb[(~local_mlb["team"].str.contains("Division")) & (local_mlb["year"].eq(2018))]
    local_mlb["team"] = local_mlb["team"].replace("\s\(.*\)", "", regex=True).str.replace("*", "")
    local_mlb[["W", "L"]] = local_mlb[["W", "L"]].astype(int)
    local_mlb["win_loss"] = local_mlb["W"] / (local_mlb["W"] + local_mlb["L"])
    local_mlb = local_mlb[["team", "win_loss"]]
    # print(local_mlb.head(30))

    local_cities = cities.copy()
    local_cities = local_cities.replace(r"(\[.*\])", "", regex=True).replace(r"(\s*\—\s*)", "", regex=True).replace("", np.nan)
    
    def team_list(team_string):
        team_pattern = "((?:[0-9]+|[A-Z])[a-z]+(?:\s(?:[0-9]+|[A-Z])[a-z]+)*)"
        return list(re.findall(team_pattern, team_string))
    
    local_cities["MLB"] = local_cities["MLB"].apply(lambda x: team_list(x) if not pd.isna(x) else np.nan)
    local_cities = local_cities.explode("MLB").reset_index(drop=True)
    local_cities.drop(["NFL", "NBA", "NHL"], axis=1, inplace=True)
    local_cities.dropna(subset=["MLB"], how="all", inplace=True, axis=0)
    # print(local_cities)
    
    known_teams = list(local_cities[~pd.isna(local_cities["MLB"])]["MLB"])
    local_mlb["team_name"] = local_mlb["team"].apply(lambda x: [y for y in known_teams if y in x])
    local_mlb["team_name"] = local_mlb["team_name"].apply(lambda x: x[0])
    
    merged_df = local_cities.merge(local_mlb, left_on="MLB", right_on="team_name", how="left")
    merged_df["Population (2016 est.)[8]"] = merged_df["Population (2016 est.)[8]"].astype(int)
    
    result_df = merged_df.groupby("Metropolitan area").agg({"Population (2016 est.)[8]": "mean", "win_loss": "mean"})
    # print(merged_df[["Metropolitan area", "Population (2016 est.)[8]", "win_loss"]])
    # print(result_df)
    
    population_by_region = result_df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = result_df["win_loss"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    local_nfl = nfl_df.copy()
    local_nfl = local_nfl[(~local_nfl["team"].str.contains("AFC|NFC")) & (local_nfl["year"].eq(2018))]
    local_nfl["team"] = local_nfl["team"].replace("\s\(.*\)", "", regex=True).str.replace("\+|\*", "")
    local_nfl[["W", "L"]] = local_nfl[["W", "L"]].astype(int)
    local_nfl["win_loss"] = local_nfl["W"] / (local_nfl["W"] + local_nfl["L"])
    local_nfl = local_nfl[["team", "win_loss"]]
    # print(local_nfl.head(30))

    local_cities = cities.copy()
    local_cities = local_cities.replace(r"(\[.*\])", "", regex=True).replace(r"(\s*\—\s*)", "", regex=True).replace("", np.nan)
    
    def team_list(team_string):
        team_pattern = "((?:[0-9]+|[A-Z])[a-z]+(?:\s(?:[0-9]+|[A-Z])[a-z]+)*)"
        return list(re.findall(team_pattern, team_string))
    
    local_cities["NFL"] = local_cities["NFL"].apply(lambda x: team_list(x) if not pd.isna(x) else np.nan)
    local_cities = local_cities.explode("NFL").reset_index(drop=True)
    local_cities.drop(["MLB", "NBA", "NHL"], axis=1, inplace=True)
    local_cities.dropna(subset=["NFL"], how="all", inplace=True, axis=0)
    # print(cities)
    
    known_teams = list(local_cities[~pd.isna(local_cities["NFL"])]["NFL"])
    local_nfl["team_name"] = local_nfl["team"].apply(lambda x: [y for y in known_teams if y in x])
    local_nfl["team_name"] = local_nfl["team_name"].apply(lambda x: x[0])
    
    merged_df = local_cities.merge(local_nfl, left_on="NFL", right_on="team_name", how="left")
    merged_df["Population (2016 est.)[8]"] = merged_df["Population (2016 est.)[8]"].astype(int)
    
    result_df = merged_df.groupby("Metropolitan area").agg({"Population (2016 est.)[8]": "mean", "win_loss": "mean"})
    # print(merged_df[["Metropolitan area", "Population (2016 est.)[8]", "win_loss"]])
    # print(result_df)
    
    population_by_region = result_df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = result_df["win_loss"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    local_nhl = nhl_df.copy()
    local_nhl = local_nhl[(~local_nhl["team"].str.contains("Division")) & (local_nhl["year"].eq(2018))]
    local_nhl["team"] = local_nhl["team"].str.replace("*", "")
    local_nhl[["W", "L"]] = local_nhl[["W", "L"]].astype(int)
    local_nhl["win_loss"] = local_nhl["W"] / (local_nhl["W"] + local_nhl["L"])
    local_nhl = local_nhl[["team", "win_loss"]]
    #print(local_nhl.head(30))
    
    local_nba = nba_df.copy()
    local_nba = local_nba[(~local_nba["team"].str.contains("Division")) & (local_nba["year"].eq(2018))]
    local_nba["team"] = local_nba["team"].replace("\s\(.*\)", "", regex=True).str.replace("*", "")
    local_nba[["W", "L"]] = local_nba[["W", "L"]].astype(int)
    local_nba["win_loss"] = local_nba["W"] / (local_nba["W"] + local_nba["L"])
    local_nba = local_nba[["team", "win_loss"]]
    # print(local_nba.head(30))

    local_mlb = mlb_df.copy()
    local_mlb = local_mlb[(~mlb_df["team"].str.contains("Division")) & (local_mlb["year"].eq(2018))]
    local_mlb["team"] = local_mlb["team"].replace("\s\(.*\)", "", regex=True).str.replace("*", "")
    local_mlb[["W", "L"]] = local_mlb[["W", "L"]].astype(int)
    local_mlb["win_loss"] = local_mlb["W"] / (local_mlb["W"] + local_mlb["L"])
    local_mlb = local_mlb[["team", "win_loss"]]
    # print(local_mlb.head(30))

    local_nfl = nfl_df.copy()
    local_nfl = local_nfl[(~local_nfl["team"].str.contains("AFC|NFC")) & (local_nfl["year"].eq(2018))]
    local_nfl["team"] = local_nfl["team"].replace("\s\(.*\)", "", regex=True).str.replace("\+|\*", "")
    local_nfl[["W", "L"]] = local_nfl[["W", "L"]].astype(int)
    local_nfl["win_loss"] = local_nfl["W"] / (local_nfl["W"] + local_nfl["L"])
    local_nfl = local_nfl[["team", "win_loss"]]
    # print(local_nfl.head(30))

    sport_df_dict = {
        "NFL": local_nfl,
        "NBA": local_nba,
        "NHL": local_nhl,
        "MLB": local_mlb
    }
        
    local_cities = cities.copy()
    local_cities = local_cities.replace(r"(\[.*\])", "", regex=True).replace(r"(\s*\—\s*)", "", regex=True).replace("", np.nan)
    
    def team_list(team_string):
        team_pattern = "((?:[0-9]+|[A-Z])[a-z]+(?:\s(?:[0-9]+|[A-Z])[a-z]+)*)"
        return list(re.findall(team_pattern, team_string))
    
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    full_df = pd.DataFrame(index=local_cities["Metropolitan area"])

    for sport in sports:
        sport_cities = local_cities.copy()
        sport_cities[sport] = sport_cities[sport].apply(lambda x: team_list(x) if not pd.isna(x) else np.nan)
        sport_cities = sport_cities.explode(sport).reset_index(drop=True)
        sport_cities.drop([x for x in sports if x != sport], axis=1, inplace=True)
        sport_cities.dropna(subset=[sport], how="all", inplace=True, axis=0)
        
        known_teams = list(sport_cities[~pd.isna(sport_cities[sport])][sport])
        sport_df_dict[sport]["team_name"] = sport_df_dict[sport]["team"].apply(lambda x: [y for y in known_teams if y in x])
        sport_df_dict[sport]["team_name"] = sport_df_dict[sport]["team_name"].apply(lambda x: x[0])
        
        sport_cities = sport_cities.merge(sport_df_dict[sport], left_on=sport, right_on="team_name", how="left")
        result_df = sport_cities.groupby("Metropolitan area").agg({"win_loss": "mean"})

        full_df = full_df.merge(result_df, left_index=True, right_index=True, how="outer")
        full_df.rename({"win_loss": sport}, axis=1, inplace=True)
        
    # print(full_df)
    p_values_dict = {}
    for sport1 in sports:
        p_values_dict[sport1] = {}
        sport1_df = full_df.dropna(subset=[sport1], axis=0, how="all")
        for sport2 in sports:
            if sport1 == sport2:
                p_values_dict[sport1][sport2] = np.nan
            else:
                sport12_df = sport1_df.dropna(subset=[sport2], axis=0, how="all")
                p_values_dict[sport1][sport2] = stats.ttest_rel(sport12_df[sport1], sport12_df[sport2])[1]
    
    p_values = pd.DataFrame.from_dict(p_values_dict)

    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
